##### Imports & setup

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize, minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import seaborn as sns

# Feature names (without linear features):
feature_names = ['mean', 'weightedMean', 'std', 'median', 'amplitude', 'beyond1Std', 'cusum', 'IPR10',
                 'kurtosis', 'MPR40_5', 'MPR20_10', 'maxSlope', 'medianAbsDev', 'medianBRP10',
                 'percentAmplitude', 'meanVariance', 'andersonDarlingNorm', 'chi2', 'skew', 'stetsonK']

## Data preparation

### Getting data

In [2]:
# The following paths should be changed accordingly. The data can be saved in a csv from the clean_data.ipynb notebook.
positive_g = pd.read_csv('../../data/clean_data/positive_class_g.csv', index_col=0)
positive_r = pd.read_csv('../../data/clean_data/positive_class_r.csv', index_col=0)
negative_g = pd.read_csv('../../data/clean_data/negative_class_g.csv', index_col=0)
negative_r = pd.read_csv('../../data/clean_data/negative_class_r.csv', index_col=0)

# Adding labels for each class:
positive_g['class'] = 'positive'
positive_r['class'] = 'positive'
negative_g['class'] = 'negative'
negative_r['class'] = 'negative'

# Normalizing features:
g_set = pd.concat([positive_g, negative_g]) # Grouping positive & negative together so that they are then normalized the same way.
g_set = normalize(g_set[feature_names], axis=0)
positive_g[feature_names] = g_set[:len(positive_g)]
negative_g[feature_names] = g_set[len(positive_g):]

### Organizing samples & train-test split

Ignoring the two filters, we have two main datasets:
1. The positive class, which contains alerts form known magnetic cataclysmic variable stars.
2. The negative class, which contains alerts from objects that are not in the positive class or that are unknown.

These two classes live in a ~20 dimensional feature space, and we hope that the positive class is somewhat clustered in that space. If that is the case, the nearest neighbors of positive class objects should also be of the positive class.

Since the positive class is poorly represented compared to the negative one, we first try here the nearest neighbors algorithm on a dataset composed of an equal number of alerts from the positive and negative classes. Then, the number of alerts from the negative class is increased by a factor two and then three.

**The train and test samples should contain different objects**

In [3]:
# IDs of unique objects in each class:
positive_g_Ids = np.unique(positive_g['objectId'])
positive_r_Ids = np.unique(positive_r['objectId'])
negative_g_Ids = np.unique(negative_g['objectId'])
negative_r_Ids = np.unique(negative_r['objectId'])

# Number of unique objects in each class:
print(f'There are {len(positive_g_Ids)} unique objects in the positive class (g filter).')
print(f'There are {len(positive_r_Ids)} unique objects in the positive class (r filter).')
print(f'There are {len(negative_g_Ids)} unique objects in the negative class (g filter).')
print(f'There are {len(negative_r_Ids)} unique objects in the negative class (r filter).')

There are 69 unique objects in the positive class (g filter).
There are 67 unique objects in the positive class (r filter).
There are 101035 unique objects in the negative class (g filter).
There are 93551 unique objects in the negative class (r filter).


In [4]:
np.random.seed(42)

# Splitting the positive class with 70% of objects in the train set and 30% in the test:
positive_g_Ids_train, positive_g_Ids_test = train_test_split(positive_g_Ids, train_size=.7)
positive_g_train = positive_g[positive_g['objectId'].isin(positive_g_Ids_train)]
positive_g_test = positive_g[positive_g['objectId'].isin(positive_g_Ids_test)]

# Splitting the negative class with 70% of objects in the train set and 30% in the test:
negative_g_Ids_train, negative_g_Ids_test = train_test_split(negative_g_Ids, train_size=.7)
negative_g_train = negative_g[negative_g['objectId'].isin(negative_g_Ids_train)]
negative_g_test = negative_g[negative_g['objectId'].isin(negative_g_Ids_test)]

# Sampling x alerts in the negative train set (x = nb of alerts in the positive train set):
negative_g_train_indices1 = np.random.choice(negative_g_train.index, len(positive_g_train), replace=False)
negative_g_train1 = negative_g_train.loc[negative_g_train_indices1]

# Sampling 2x alerts in the negative train set
negative_g_train_indices2 = np.random.choice(negative_g_train.index, 2*len(positive_g_train), replace=False)
negative_g_train2 = negative_g_train.loc[negative_g_train_indices2]

# Sampling 3x alerts in the negative train set
negative_g_train_indices3 = np.random.choice(negative_g_train.index, 3*len(positive_g_train), replace=False)
negative_g_train3 = negative_g_train.loc[negative_g_train_indices3]

# Assembling the final train sets:
train_g1 = pd.concat([positive_g_train, negative_g_train1]).reset_index()
train_g2 = pd.concat([positive_g_train, negative_g_train2]).reset_index()
train_g3 = pd.concat([positive_g_train, negative_g_train3]).reset_index()
train_g4 = pd.concat([positive_g_train, negative_g_train]).reset_index() # Using the whole negative class

## Finding nearest neighbors

### Wrapper

In [5]:
def NN_wrapper(train: pd.DataFrame, test: pd.DataFrame, show_counts: bool = False, **NN_kwargs) -> tuple[pd.DataFrame, float]:
    """Wrapper for the NearestNeighbors class from scikit-learn.
    This function takes the training and testing sets and prints out the propotion of neighbors that are of the positive and negative class.

    Args:
        train (pd.DataFrame): Training set (a combination of positive and negative class objects).
        test (pd.DataFrame): Testing set (positive class objects only, and different from those of the training set).
        show_counts (bool, optional): Wether to show the number of neighbors in each class or not. Defaults to False.
        NN_kwargs: Keyword arguments for the NearestNeighbors class from scickit-learn.

    Returns:
        tuple (neighbors, accuracy):
            - neighbors (pd.DataFrame): The neighbors of the test set objects.
            - accuracy (float): The ratio of neighbors that are of the positive class.
    """

    default_kwargs: dict = {}
    default_kwargs.update(NN_kwargs)

    # Fitting the NearestNeighbors model:
    neigh = NearestNeighbors(**default_kwargs)
    neigh.fit(train[feature_names])

    # Finding the nearest neighbors of the test set objects:
    neighbors_indices = neigh.kneighbors(test[feature_names], return_distance=False)
    neighbors = train.loc[neighbors_indices.flatten()]
    
    if show_counts:
        print(neighbors['class'].value_counts())

    # Calculating the accuracy of the model:
    accuracy = (neighbors['class'] == 'positive').sum() / len(neighbors)
    # print(f'\nPercentage of positive class samples in the test set that have a positive class nearest neighbor: {accuracy*100:.0f} %')

    return neighbors, accuracy

### Results

In [6]:
def explore_params(train, test):
    K: list[int] = [1, 5, 10, 15, 20]
    leaf_sizes: list[int] = [2, 5, 10, 20, 30, 40, 50]
    algorithms: list[str] = ['ball_tree', 'kd_tree', 'brute']

    max_accuracy: dict = {'accuracy': 0, 'algo': 'none', 'k': 0, 'leaf_size': 0}

    for algo in algorithms:
        print('\n'+algo+':')
        for k in K:
            print(f'{k=}:')
            for leaf_size in leaf_sizes:
                print(f'{leaf_size=}:')
                _, accuracy = NN_wrapper(train, test, n_neighbors=k, algorithm=algo, leaf_size=leaf_size)
                if accuracy > max_accuracy['accuracy']:
                    max_accuracy = {'accuracy': accuracy, 'algo': algo, 'k': k, 'leaf_size': leaf_size}
                print(accuracy)

    print(f'{max_accuracy=}')
    
    return

#### Train sample positive-negative ratio: 1-1

In [7]:
explore_params(train_g1, positive_g_test)


ball_tree:
k=1:
leaf_size=2:
0.5944609297725024
leaf_size=5:
0.5944609297725024
leaf_size=10:
0.5944609297725024
leaf_size=20:
0.5944609297725024
leaf_size=30:
0.5944609297725024
leaf_size=40:
0.5944609297725024
leaf_size=50:
0.5944609297725024
k=5:
leaf_size=2:
0.6365974282888229
leaf_size=5:
0.6365974282888229
leaf_size=10:
0.6365974282888229
leaf_size=20:
0.6365974282888229
leaf_size=30:
0.6365974282888229
leaf_size=40:
0.6365974282888229
leaf_size=50:
0.6365974282888229
k=10:
leaf_size=2:
0.636003956478734
leaf_size=5:
0.636003956478734
leaf_size=10:
0.636003956478734
leaf_size=20:
0.636003956478734
leaf_size=30:
0.636003956478734
leaf_size=40:
0.636003956478734
leaf_size=50:
0.636003956478734
k=15:
leaf_size=2:
0.6379162545334652
leaf_size=5:
0.6379162545334652
leaf_size=10:
0.6379162545334652
leaf_size=20:
0.6379162545334652
leaf_size=30:
0.6379162545334652
leaf_size=40:
0.6379162545334652
leaf_size=50:
0.6379162545334652
k=20:
leaf_size=2:
0.6368941641938675
leaf_size=5:
0.6368

#### Train sample positive-negative ratio: 1-2

In [8]:
explore_params(train_g2, positive_g_test)


ball_tree:
k=1:
leaf_size=2:
0.4826904055390702
leaf_size=5:
0.4826904055390702
leaf_size=10:
0.4826904055390702
leaf_size=20:
0.4826904055390702
leaf_size=30:
0.4826904055390702
leaf_size=40:
0.4826904055390702
leaf_size=50:
0.4826904055390702
k=5:
leaf_size=2:
0.5266073194856578
leaf_size=5:
0.5266073194856578
leaf_size=10:
0.5266073194856578
leaf_size=20:
0.5266073194856578
leaf_size=30:
0.5266073194856578
leaf_size=40:
0.5266073194856578
leaf_size=50:
0.5266073194856578
k=10:
leaf_size=2:
0.5275964391691395
leaf_size=5:
0.5275964391691395
leaf_size=10:
0.5275964391691395
leaf_size=20:
0.5275964391691395
leaf_size=30:
0.5275964391691395
leaf_size=40:
0.5275964391691395
leaf_size=50:
0.5275964391691395
k=15:
leaf_size=2:
0.5233761951862842
leaf_size=5:
0.5233761951862842
leaf_size=10:
0.5233761951862842
leaf_size=20:
0.5233761951862842
leaf_size=30:
0.5233761951862842
leaf_size=40:
0.5233761951862842
leaf_size=50:
0.5233761951862842
k=20:
leaf_size=2:
0.5223541048466864
leaf_size=5:

#### Train sample positive-negative ratio: 1-3

In [9]:
explore_params(train_g3, positive_g_test);


ball_tree:
k=1:
leaf_size=2:
0.42136498516320475
leaf_size=5:
0.42136498516320475
leaf_size=10:
0.42136498516320475
leaf_size=20:
0.42136498516320475
leaf_size=30:
0.42136498516320475
leaf_size=40:
0.42136498516320475
leaf_size=50:
0.42136498516320475
k=5:
leaf_size=2:
0.46963402571711177
leaf_size=5:
0.46963402571711177
leaf_size=10:
0.46963402571711177
leaf_size=20:
0.46963402571711177
leaf_size=30:
0.46963402571711177
leaf_size=40:
0.46963402571711177
leaf_size=50:
0.46963402571711177
k=10:
leaf_size=2:
0.4664688427299703
leaf_size=5:
0.4664688427299703
leaf_size=10:
0.4664688427299703
leaf_size=20:
0.4664688427299703
leaf_size=30:
0.4664688427299703
leaf_size=40:
0.4664688427299703
leaf_size=50:
0.4664688427299703
k=15:
leaf_size=2:
0.46416089680184636
leaf_size=5:
0.46416089680184636
leaf_size=10:
0.46416089680184636
leaf_size=20:
0.46416089680184636
leaf_size=30:
0.46416089680184636
leaf_size=40:
0.46416089680184636
leaf_size=50:
0.46416089680184636
k=20:
leaf_size=2:
0.46112759

#### Train sample positive-negative ratio: 1-full

In [10]:
explore_params(train_g4, positive_g_test)


ball_tree:
k=1:
leaf_size=2:
0.09099901088031652
leaf_size=5:
0.09099901088031652
leaf_size=10:
0.09099901088031652
leaf_size=20:
0.09099901088031652
leaf_size=30:
0.09099901088031652
leaf_size=40:
0.09099901088031652
leaf_size=50:
0.09099901088031652
k=5:
leaf_size=2:
0.08051434223541049
leaf_size=5:
0.08051434223541049
leaf_size=10:
0.08051434223541049
leaf_size=20:
0.08051434223541049
leaf_size=30:
0.08051434223541049
leaf_size=40:
0.08051434223541049
leaf_size=50:
0.08051434223541049
k=10:
leaf_size=2:
0.08120672601384768
leaf_size=5:
0.08120672601384768
leaf_size=10:
0.08120672601384768
leaf_size=20:
0.08120672601384768
leaf_size=30:
0.08120672601384768
leaf_size=40:
0.08120672601384768
leaf_size=50:
0.08120672601384768
k=15:
leaf_size=2:
0.08084404879657105
leaf_size=5:
0.08084404879657105
leaf_size=10:
0.08084404879657105
leaf_size=20:
0.08084404879657105
leaf_size=30:
0.08084404879657105
leaf_size=40:
0.08084404879657105
leaf_size=50:
0.08084404879657105
k=20:
leaf_size=2:
0.0